In [ ]:
import constants
import pandas  as pd
from tqdm import tqdm
import hashlib
import collections
import itertools
import functools
import seaborn as sns
import pickle
import safer
import matplotlib.pyplot as plt
import math
import random
import numpy as np

In [ ]:
sns.set_style(style="whitegrid")

In [ ]:
extended = False
constants.set_crawl_source(constants.CRAWL_SOURCE_EXTENDED if extended else constants.CRAWL_SOURCE_CHICAGO)

In [ ]:
experiment = 2
if not extended:
    experiment = 1

In [ ]:
tqdm.pandas()

In [ ]:
df_all = pd.read_pickle(constants.LONG_DATA_FILE)

In [ ]:
df_all["crawl_id"].unique()

In [ ]:
df_all = df_all[df_all.crawl_id.isin(constants.CRAWL_ORDER)]

In [ ]:
df_all["crawl_id"].unique()

In [ ]:
df_all["crawl_number"] = df_all["crawl_id"].apply(lambda x: constants.CRAWL_NUMBER[x])
df = df_all[df_all.experiment == experiment]
df = df.sort_values("crawl_number")

In [ ]:
df["author_id"] = pd.concat([df[df.user_page_url.notnull()].user_page_url.str.slice(start=len("/user_details?userid=")),df[df.data_hovercard_id.notnull()].data_hovercard_id])

In [ ]:
stats_df = pd.read_pickle(constants.RECALSSIFICATION_DATA_FILE)

# Sampling functions 

In [ ]:
prg = random.Random(0)

In [ ]:
pattern_class_chars = {True: "-", False: "+", None: ""} #True means not recommended, which is more intuitively a '-'
def get_pattern(reclass_order):
    pattern = []
    prev_c = None
    for c in reclass_order:
        if c is None:
            continue
        
        if c != prev_c:
            pattern.append(c)
            prev_c = c
    return "".join(pattern_class_chars[x] for x in pattern)

order_class_chars = {True: "-", False: "+", None: "_"} #True means not recommended, which is more intuitively a '-'
def get_order(reclass_order):
    pattern = []
    prev_c = None
    for c in reclass_order:
        if c is None:
            continue
        
        pattern.append(c)
    return "".join(order_class_chars[x] for x in pattern)

In [ ]:
def sample_case(stats_df=stats_df,seed=0,count=1):
    sample = stats_df.sample(n=count,random_state=seed)
    for rowid, row in sample.iterrows():
        members = df.loc[row.members]
        print(f"""Reclassificaiton pattern: {get_pattern( row.reclassification_order)}
Reclassificaiton order: {get_order( row.reclassification_order)}
Rating: {members.iloc[0].rating}
Date: {members.iloc[0].date.strftime('%Y/%m/%d')}
Name: {members.iloc[0].user_name}

{members.iloc[0].content}

""")
        display(members[["flagged","rating","user_friends","user_review_count","user_photos","elite", "user_name","user_location", "crawl_id"]])
        print("\n------------------------------------------------------------\n")

In [ ]:
list(df)

In [ ]:
sample_case()

# Case studies

In [ ]:
stats_df.reclassification_order.apply(lambda x: 0 if type(x) is float else len(x)).unique()

In [ ]:
no_reclass_reviews = stats_df[stats_df.reclassification_swaps == 0]

In [ ]:
reclass_reviews = stats_df[stats_df.reclassification_swaps >= 1]

In [ ]:
for i in range(50):
    sample_case(reclass_reviews,i)
    print("----------------------------------------------------------------")

In [ ]:
multiple_reclass_reviews = stats_df[stats_df.reclassification_swaps >= 2]

In [ ]:
for i in range(50):
    sample_case(multiple_reclass_reviews,i)
    print("----------------------------------------------------------------")